In [95]:
import csv
import numpy as np
import pandas
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook

In [96]:
# Column names from our CSV
colNames = ['S1', 'C1', 'S2', 'C2', 'S3', 'C3', 'S4', 'C4', 'S5', 'C5', 'CLASS']

# Read CSV file using pandas
data = pandas.read_csv('data.csv', names=colNames)

# Extract all data from the CLASS column
hands = data.CLASS.tolist()

# Remove the first element 
hands.pop(0)

' CLASS'

Loading BokehJS ...